# 에이전트 평가

이 섹션에서는 에이전트 시스템의 평가에 대해 살펴보겠습니다. 에이전트 시스템은 여러 하위 구성 요소로 이루어진 복잡한 구조입니다. Lab 3에서는 두 개의 도구를 조율하는 단일 에이전트를 살펴보았습니다. Lab 4에서는 상위 수준의 라우터 에이전트가 여러 에이전트 하위 시스템을 조정하는 더 정교한 다층 에이전트 시스템을 보여주었습니다.

에이전트 시스템의 상호 의존적이고 잠재적으로 중첩된 특성의 직접적인 의미는 평가가 거시적 및 미시적 수준에서 모두 이루어질 수 있다는 것입니다. 이는 전체 시스템을 평가하거나(거시적 관점) 각 개별 하위 구성 요소를 평가한다(미시적 관점)는 것을 의미합니다. 중첩된 시스템의 경우 이는 모든 추상화 수준에 적용됩니다.

<img src="./images/macro-micro-eval.png" width="100%" alt='macro-micro-eval.png'/> 

일반적으로 평가는 거시적 수준에서 시작됩니다. 대부분의 경우 거시적 수준의 긍정적인 평가 결과는 충분한 에이전트 성능을 나타냅니다. 거시적 수준의 성능 평가가 불충분하거나 좋지 않은 결과를 보이는 경우, 미시적 수준의 평가는 성능 지표를 분해하고 결과를 특정 하위 구성 요소에 귀속시키는 데 도움이 될 수 있습니다.

이 실습에서는 먼저 거시적 수준의 에이전트 성능 평가를 살펴보겠습니다. 그런 다음 하위 수준 작업으로서 도구 사용을 평가할 것입니다. 평가 프로세스에 집중하기 위해 Lab 3에서 구성한 단일 에이전트를 활용하여 에이전트 시스템을 단순하게 유지하겠습니다.

## 에이전트 설정

이전 섹션에서 다룬 바와 같이 Lab 3에서 구축한 에이전트를 재사용할 것입니다. 이 에이전트는 휴가지 찾기를 돕도록 설계된 도구에 접근할 수 있습니다. 질문을 하고 다양한 도구를 사용하는 것을 관찰하며 의미 있는 대화를 나눌 수 있습니다.

필요한 패키지를 설치하는 것부터 시작하겠습니다.

<div class="alert alert-block alert-warning">
<b>주의:</b> 이 노트북을 실행하기 위해서 커널을 종료(shut down) 후 다시 시작해주세요
</div>

In [ ]:
%pip install -U langchain-community langgraph langchain-chroma langchain_aws pandas ragas==0.2.6 faiss-cpu==1.8.0 pypdf rapidfuzz
%pip install -U  pandas ragas==0.2.6 faiss-cpu==1.8.0 pypdf -q

In [ ]:
!pip install rapidfuzz --force-reinstall -q

### 유틸리티 함수 파트 1 - 단일 에이전트 가져오기

이 노트북에서 깔끔하고 집중된 접근 방식을 유지하기 위해 에이전트 생성 로직을 `utils.py` 모듈로 옮겼습니다. `create_agent` 함수는 Lab 3에서 개발한 간단한 ReAct 에이전트의 생성 프로세스를 복제합니다.

In [3]:
from utils import create_agent
agent_executor = create_agent()

`create_agent` 함수는 Lab 3 시나리오의 에이전트를 나타내는 `CompiledStateGraph` 객체를 반환합니다. 
이제 이 그래프를 시각화해보겠습니다.

In [4]:
from IPython.display import Image, display

display(Image(agent_executor.get_graph().draw_mermaid_png()))

이제 에이전트를 평가할 준비가 되었습니다!

## ragas 라이브러리를 사용한 에이전트 평가

이 섹션에서는 ragas 라이브러리를 사용하여 에이전트 시스템을 평가하는 정교한 방법을 살펴보겠습니다. Lab 3의 휴가지 추천 에이전트를 기반으로, 상위 수준(거시적)과 하위 수준(미시적) 평가 접근 방식을 모두 구현할 것입니다.

ragas는 에이전트 시스템에 특히 중점을 둔 대규모 언어 모델(LLM) 애플리케이션 평가를 위한 특수 도구를 제공합니다. 다음 두 가지 주요 평가 차원에 초점을 맞출 것입니다:

1. [상위 수준 에이전트 정확도](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/agents/#agent-goal-accuracy):
   - 에이전트 목표 정확도(참조 포함): 주석이 달린 참조 응답과 결과를 비교하여 에이전트가 지정된 목표를 얼마나 잘 달성하는지 측정
   - 에이전트 목표 정확도(참조 없음): 사용자 상호작용에서 원하는 결과를 추론하여 목표 달성을 평가

2. [하위 수준 도구 사용](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/agents/#tool-call-accuracy):
   - 도구 호출 정확도: 실제 도구 호출을 참조 도구 호출과 비교하여 에이전트가 적절한 도구를 식별하고 활용하는 능력을 평가
   - 지표는 0에서 1 사이이며, 높은 값이 더 나은 성능을 나타냄

이 구조화된 평가 접근 방식을 통해 휴가지 추천 에이전트의 성능을 시스템 수준과 구성 요소 수준 모두에서 종합적으로 평가할 수 있습니다. Lab 3의 단일 에이전트에 초점을 맞춤으로써 중첩된 에이전트 시스템의 추가적인 복잡성 없이 이러한 평가 기법을 명확하게 보여줄 수 있습니다.

휴가 관련 쿼리와 도구 상호작용을 처리하는 에이전트의 효과성을 분석하기 위해 이러한 평가 방법을 구현해 보겠습니다.

### 유틸리티 함수 파트 2 - 메시지 형식 변환

우리의 단일 에이전트는 LangChain/LangGraph 프레임워크를 사용하여 구축되었습니다. LangChain은 에이전트 시스템 내의 다양한 유형의 통신을 처리하기 위해 여러 [메시지 객체](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/)를 정의합니다. LangChain 문서에 따르면 다음과 같습니다:

- HumanMessage: 사용자의 메시지를 나타냅니다. 일반적으로 내용으로만 구성됩니다.
- AIMessage: 모델의 메시지를 나타냅니다. Amazon Bedrock 도구 호출을 사용하는 경우 tool_calls와 같은 additional_kwargs가 포함될 수 있습니다.
- ToolMessage: 도구 호출의 결과를 나타냅니다. 역할과 내용 외에도 이 메시지에는 이 결과를 생성하기 위해 호출된 도구의 호출 ID를 전달하는 `tool_call_id` 매개변수가 있습니다.

마찬가지로 ragas 라이브러리는 자체 메시지 래퍼 객체를 구현합니다:

- [HumanMessage](https://docs.ragas.io/en/latest/references/evaluation_schema/?h=aimessage#ragas.messages.HumanMessage): 사용자의 메시지를 나타냅니다.
- [AIMessage](https://docs.ragas.io/en/latest/references/evaluation_schema/?h=aimessage#ragas.messages.AIMessage): AI의 메시지를 나타냅니다.
- [ToolMessage](https://docs.ragas.io/en/latest/references/evaluation_schema/?h=aimessage#ragas.messages.ToolMessage): 도구의 메시지를 나타냅니다.
- [ToolCall](https://docs.ragas.io/en/latest/references/evaluation_schema/?h=aimessage#ragas.messages.ToolCall): 이름과 인수가 포함된 도구 호출을 나타냅니다(일반적으로 도구 호출이 사용될 때 `AIMessage` 내에 포함됨)

LangGraph 에이전트가 생성한 대화 흐름을 평가하기 위해서는 이 두 메시지 타입 시스템 간에 변환이 필요합니다. 편의를 위해 `utils.py` 모듈에 `convert_message_langchian_to_ragas` 함수를 구현했습니다. 이 함수는 변환 프로세스를 원활하게 처리합니다. 프레임워크 간의 호환성을 보장하기 위해 적절한 별칭이 할당된 메시지 래퍼 객체와 함께 이를 가져올 수 있습니다.

In [5]:
from utils import create_agent, convert_message_langchain_to_ragas

from langchain_core.messages import HumanMessage as LCHumanMessage
from langchain_core.messages import AIMessage as LCAIMessage
from langchain_core.messages import ToolMessage as LCToolMessage


from ragas.messages import Message as RGMessage
from ragas.messages import HumanMessage as RGHumanMessage
from ragas.messages import AIMessage as RGAIMessage
from ragas.messages import ToolMessage as RGToolMessage
from ragas.messages import ToolCall as RGToolCall

일반적으로 여러 번의 대화를 평가하므로 메시지 배열을 처리할 수 있는 헬퍼 함수를 구현하겠습니다. 이 함수를 통해 여러 대화 교환을 원활하게 처리하고 분석할 수 있습니다.

In [6]:
def convert_messages(response):
    return list(map((lambda m: convert_message_langchain_to_ragas(m)), response['messages']))

평가를 위해 서로 다른 사용자 프로필을 대표하는 두 가지 시나리오를 살펴보겠습니다:

1. **Andrew Macdonald - 여행 이력이 있는 사용자**
   * 파리에 거주하는 62세
   * 여행 이력 데이터베이스에 존재하며 user_id 918로 로그인됨
   * 이전 여행 기록을 통해 정확한 맞춤형 추천이 가능
   * 원활한 대화 흐름이 예상됨

2. **Jane Doe - 첫 방문 사용자**
   * 여행 추천 시스템과의 이전 상호작용이 없음
   * 에이전트가 창의적인 추천 전략에 의존해야 함
   * 정보를 수집하고 관련 제안을 제공하는 시스템의 능력을 테스트함
   * 약간 더 탐색적인 대화 흐름을 경험할 수 있음

이러한 시나리오는 서로 다른 사용자 유형과 상호작용 패턴에 걸쳐 우리의 여행 에이전트의 성능을 평가하는 데 도움이 될 것입니다. 이제 시스템의 효과성을 평가하기 위해 이러한 대화 흐름을 실행해 보겠습니다.

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"user_id": 918}}
response_andrew = agent_executor.invoke(
        {"messages": [HumanMessage(content="Please suggest me a good vacation destination alongside some details.")]},
        config,
    )
response_andrew

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {}}
response_jane = agent_executor.invoke(
        {"messages": [HumanMessage(content="Suggest me a good vacation destination. I love beaches!")]},
        config,
    )
response_jane

이제 Andrew와 Jane의 에이전트 대화를 수집했으므로 이를 LangChain의 메시지 형식에서 ragas 메시지 형식으로 변환할 수 있습니다. 이 변환을 위해 이전에 정의한 `convert_messages` 함수를 사용할 것입니다.

In [9]:
rg_messages_andrew = convert_messages(response_andrew)
for m in rg_messages_andrew:
    print(m.pretty_repr())
    print("-----------")

In [10]:
rg_messages_jane = convert_messages(response_jane)
for m in rg_messages_jane:
    print(m.pretty_repr())
    print("-----------")

이제 대화 흐름이 적절하게 포맷되었으므로 실제 평가 단계를 진행할 수 있습니다.

### 에이전트 목표 정확도

에이전트 목표 정확도는 LLM이 사용자 목표를 식별하고 달성하는 정도를 평가하도록 설계된 지표입니다. 1은 목표 달성 성공을, 0은 실패를 나타내는 이진 지표입니다. 평가는 지표 계산 전에 정의하고 구성해야 하는 평가자 LLM을 사용하여 수행됩니다.

에이전트 목표 정확도 지표는 두 가지 구별되는 변형이 있습니다:

- 참조 없는 에이전트 목표 정확도
- 참조 있는 에이전트 목표 정확도

이러한 변형을 자세히 살펴보기 전에 평가자 LLM을 설정해야 합니다. 이를 위해 Anthropic Claude 3 Haiku를 판단자로 사용할 것입니다. 이것이 이 실습에서의 선택이지만, 평가자 LLM의 선택은 항상 특정 사용 사례와 요구 사항에 따라 조정되어야 합니다.

In [11]:
import boto3
from ragas.llms import LangchainLLMWrapper
from langchain_aws import ChatBedrockConverse

# ---- ⚠️ Update region for your AWS setup ⚠️ ----
bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")

judge_llm = LangchainLLMWrapper(ChatBedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0,
    max_tokens=None,
    client=bedrock_client,
    # other params...
))

#### 참조 없는 에이전트 목표 정확도
AgentGoalAccuracyWithoutReference는 사전 정의된 참조 지점 없이 작동합니다. 대신 워크플로우 내의 인간 상호작용에서 원하는 결과를 추론하여 LLM의 성능을 평가합니다. 이 접근 방식은 명시적인 참조 결과를 사용할 수 없거나 성공 기준을 대화 맥락에서 결정할 수 있을 때 특히 유용합니다.

이 지표를 평가하기 위해 먼저 에이전트 대화를 ragas 생태계 내에서 다중 턴 에이전트 대화를 처리하도록 설계된 `MultiTurnSample` 객체에 캡슐화합니다. 다음으로 평가 지표를 구현하기 위해 `AgentGoalAccuracyWithoutReference` 객체를 초기화합니다. 마지막으로 판단자 LLM을 구성하고 세 가지 에이전트 대화에 걸쳐 평가를 실행합니다.

In [12]:
from ragas.dataset_schema import  MultiTurnSample
from ragas.messages import HumanMessage,AIMessage,ToolMessage,ToolCall
from ragas.metrics import AgentGoalAccuracyWithoutReference


sample_andrew = MultiTurnSample(user_input=rg_messages_andrew)

sample_jane = MultiTurnSample(user_input=rg_messages_jane)

scorer = AgentGoalAccuracyWithoutReference(llm=judge_llm)

In [13]:
await scorer.multi_turn_ascore(sample_andrew)

In [14]:
await scorer.multi_turn_ascore(sample_jane)

#### 참조 있는 에이전트 목표 정확도
AgentGoalAccuracyWithReference는 두 가지 주요 입력이 필요합니다: user_input과 참조 결과입니다. 이 변형은 LLM의 달성된 결과를 이상적인 결과로 작용하는 주석이 달린 참조와 비교하여 LLM의 성능을 평가합니다. 지표는 워크플로우의 끝에서 LLM의 결과가 미리 정의된 참조 결과와 얼마나 잘 일치하는지 평가하여 계산됩니다.

이 지표를 평가하기 위해 비슷한 접근 방식을 따를 것입니다. 먼저 ragas 라이브러리에서 다중 턴 에이전트 대화를 관리하도록 특별히 설계된 `MultiTurnSample` 객체 내에 에이전트 대화를 캡슐화합니다. 이 평가를 위해서는 판단자의 평가 기준으로 작용할 주석이 달린 참조를 제공해야 합니다. 그런 다음 평가 지표를 구현하기 위해 `AgentGoalAccuracyWithReference` 객체를 초기화합니다. 이를 통해 평가자 LLM을 설정합니다. 그런 다음 정의된 기준에 대한 성능을 측정하기 위해 세 가지 에이전트 대화 모두에 걸쳐 평가를 수행합니다.

In [15]:
from ragas.dataset_schema import  MultiTurnSample
from ragas.messages import HumanMessage,AIMessage,ToolMessage,ToolCall
from ragas.metrics import AgentGoalAccuracyWithReference


sample_andrew = MultiTurnSample(user_input=rg_messages_andrew,
    reference="Provide a holiday suggestion alongside of detailed information about destination.")

sample_jane = MultiTurnSample(user_input=rg_messages_jane,
    reference="Provide detailed information about suggested holiday destination.")

scorer = AgentGoalAccuracyWithReference(llm=judge_llm)

In [ ]:
await scorer.multi_turn_ascore(sample_andrew)

In [ ]:
await scorer.multi_turn_ascore(sample_jane)

결과를 분석하고 각 페르소나의 에이전트와의 상호작용 패턴과의 관계를 살펴보겠습니다. 더 깊은 통찰을 얻기 위해 워크숍 그룹과 이러한 발견을 논의하는 것을 권장합니다. 에이전트 대화는 동적이고 비결정적이므로 평가 결과는 다른 실행에서 다를 수 있다는 점을 염두에 두세요. 

그러나 특정 패턴이 나타납니다:
- Andrew의 대화는 일반적으로 집중되고 목표 지향적인 접근 방식으로 인해 1.0 등급을 달성합니다
- Jane의 대화는 일반적으로 0.0으로 평가됩니다. 과거 정보가 부족하여 시스템이 단일 대화 턴에서 제안을 제공할 수 없습니다. 인간 개입 접근 방식으로 그녀의 관심사를 물어보면 이 문제를 해결할 수 있습니다.

특히 `AgentGoalAccuracyWithReference`의 경우 대화 흐름이나 참조를 조정하여 결과에 영향을 미칠 수 있다는 점에 유의하세요. 시간이 남으면 시도해보세요!

### 도구 호출 정확도

ToolCallAccuracy는 주어진 작업을 완료하는 데 필요한 도구를 식별하고 호출하는 LLM의 성능을 평가하는 데 사용할 수 있는 지표입니다. 이 지표는 주어진 작업을 완료하는 데 필요한 도구를 식별하고 호출하는 LLM의 성능을 평가하기 위해 user_input과 reference_tool_calls가 필요합니다. 이 지표는 reference_tool_calls와 AI가 수행한 도구 호출을 비교하여 계산됩니다. 따라서 이 특정 시나리오에서는 평가자 LLM이 필요하지 않습니다. 값은 0에서 1 사이이며, 높은 값이 더 나은 성능을 나타냅니다.

도구 호출 정확도 지표를 평가하기 위해 다른 프로세스를 따릅니다. 먼저 ragas 라이브러리에서 다중 턴 에이전트 대화를 처리하도록 특별히 설계된 `MultiTurnSample` 객체 내에 에이전트 대화를 캡슐화합니다. 이 평가에는 평가의 기준으로 작용하는 주석이 달린 참조 도구 호출 세트가 필요합니다. 다음으로 평가 지표를 구현하기 위해 `ToolCallAccuracy` 객체를 초기화합니다. 기본 동작은 정확한 문자열 매칭을 사용하여 도구 이름과 인수를 비교하지만, 특히 자연어 인수를 다룰 때 이것이 항상 최적이지는 않을 수 있습니다. 이를 완화하기 위해 ragas는 검색된 컨텍스트의 관련성을 더 효과적으로 결정하는 데 사용할 수 있는 다양한 NLP 거리 지표 중에서 선택할 수 있습니다. 이 실습에서는 Levenshtein, Hamming, Jaro와 같은 전통적인 문자열 거리 측정을 활용하는 `NonLLMStringSimilarity`를 사용합니다. 따라서 `arg_comparison_metric` 매개변수를 `NonLLMStringSimilarity`로 설정합니다.

In [19]:
from ragas.metrics import ToolCallAccuracy
from ragas.dataset_schema import  MultiTurnSample
from ragas.messages import HumanMessage,AIMessage,ToolMessage,ToolCall
from ragas.metrics._string import NonLLMStringSimilarity



sample_andrew = MultiTurnSample(
    user_input=rg_messages_andrew,
    reference_tool_calls=[
        ToolCall(name="compare_and_recommend_destination", args={}),
        ToolCall(name="travel_guide", args={"query": "Ljubljana"}),
    ]
)

sample_jane = MultiTurnSample(
    user_input=rg_messages_jane,
    reference_tool_calls=[
        ToolCall(name="compare_and_recommend_destination", args={}),
        ToolCall(name="travel_guide", args={"query": "Miami, Florida"}),
    ]
)

scorer = ToolCallAccuracy()
scorer.arg_comparison_metric = NonLLMStringSimilarity()

In [ ]:
await scorer.multi_turn_ascore(sample_andrew)

In [ ]:
await scorer.multi_turn_ascore(sample_jane)

결과를 분석하고 각 페르소나의 에이전트와의 상호작용 패턴과의 관계를 살펴보겠습니다. 더 깊은 통찰을 얻기 위해 워크숍 그룹과 이러한 발견을 논의하는 것을 권장합니다. 에이전트 대화는 동적이고 비결정적이므로 평가 결과는 다른 실행에서 다를 수 있다는 점을 염두에 두세요.

그러나 특정 패턴이 나타납니다:
- Andrew의 대화는 일반적으로 집중되고 목표 지향적인 접근 방식과 여행 데이터베이스에서 찾을 수 있다는 사실 때문에 매우 높은 등급을 달성합니다 - 이는 모든 도구 호출 인수를 일치시키는 데 도움이 됩니다
- Jane의 대화는 일반적으로 높지만 약간 낮은 등급을 달성합니다. 대화가 집중되고 목표 지향적이지만 여행 데이터베이스에 없습니다. 이로 인해 도구 호출 인수가 덜 결정적이 되어 특정 도시를 추천할 가능성이 줄어듭니다. 시간이 있다면 `travel_guide` 도구 호출의 참조 `query` 인수를 "beach destination"과 같이 수정해보세요. 메시지 히스토리의 도구 호출과 결과를 상관시켜보세요. 무엇을 관찰하나요?

축하합니다! 이 실습과 전체 워크숍을 성공적으로 완료했습니다. 오늘 세션에 적극적으로 참여해 주셔서 감사합니다. 다룬 주제에 대해 질문이나 명확히 하고 싶은 점이 있다면 강사에게 문의하시기 바랍니다.
